<a href="https://colab.research.google.com/github/SangMin316/2022_Dacon_Contest/blob/main/mbti_500_Linear_support_vector_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score
import pickle
import os.path
import plotly.offline as pyo
import plotly.graph_objs as go
import spacy

# Dataset

나의 구글 드라이브를 mount해준다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


데이터를 불러옵니다.

이때, 열의 이름을 'type','posts'로 지정해서 받아옵니다.


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/2022 big_data/dataset/MBTI_train.csv', names=["type","posts"])

In [ ]:
# 데이터 양상 확인하기
df.head() 

,type,posts
0,INTP,say process model list like subscriber channel...
1,INFJ,upon much manipulate retail finish like sacrif...
2,INFJ,fit yes certain bff social feel goal go know n...
3,INTJ,complete love within someone ideal joke solvea...
4,ENTJ,public strictly thing person x question person...


In [ ]:
len(df['posts'][1])

2530

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 500)
tokenizer.fit_on_texts(df['posts'][0])
word_dic = tokenizer.word_index
print(len(word_dic))


26


type에는 mbti 16종류가 있음을 확인합니다.

In [ ]:
df['type'].unique() 

array(['INTP', 'INFJ', 'INTJ', 'ENTJ', 'ENTP', 'INFP', 'ISTP', 'ISFJ',
       'ENFP', 'ISFP', 'ISTJ', 'ENFJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ'],
      dtype=object)

##null값이 있는지 확인합니다.

In [ ]:
df.isnull().sum()


type     0
posts    0
dtype: int64

# Drow mbti histogram.

In [ ]:
df_bar_chart=df.groupby('type').count()
trace1 = go.Bar(x=df_bar_chart.index, y=df_bar_chart['posts'])
data = [trace1]
layout = go.Layout(title='MBTI # Classified Posts per Type')
fig = go.Figure(data=data, layout=layout)
fig.show()

# Model

# 저희는 모든 데이터를 학습에 사용했습니다.

validation data를 따로 만들어주지 않았습니다.

data 갯수가 부족한 class들이 존재하기 때문입니다. 안그래도 적은 데이터를 train과 valdiatation으로 나누기에는 위험요소가 있다가 판단했습니다.

In [ ]:
X_train = df['posts'] # features
y_train = df['type']  # labels

# 모델을 만들어줍니다.

# vectorize
posts에 있는 단어들로 터를 만들어줍니다.

tf-idf Vectorize를 이용합니다.

전체 단어 셋에서의 빈도수를 고려해서, 특정 mbti 성격유형에서만 자주 등장하는
단어에 가중치를 높이기 위해서입니다.

In [ ]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)

In [ ]:
X_train_tfidf

<74357x172439 sparse matrix of type '<class 'numpy.float64'>'
	with 21058879 stored elements in Compressed Sparse Row format>

# Linear support vector classifier
분류 문제이기 때문에 linear support vetor classifier를 사용합니다.

In [ ]:
clf = LinearSVC()
clf.fit(X_train_tfidf, y_train)

LinearSVC()

# pipeline을 만들어줍니다.

In [ ]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

# testdata 불러와서 제출파일 만들기

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/2022 big_data/dataset/MBTI_test.csv', names = ['posts'])

불러온 데이터를 학습시켜줍니다.

In [ ]:
test = submission['posts']
predictions = text_clf.predict(test)

예측결과가 잘 나왔는지 확인해봅니다.

In [ ]:
predictions

array(['ENFP', 'ENTP', 'INTJ', ..., 'INTP', 'ENFP', 'INFP'], dtype=object)

나온 결과를 저장해줍니다.

In [ ]:
testdf = pd.DataFrame(predictions)
 
testdf.to_csv('test.csv', index=False)

In [ ]:
submission['type'] = predictions
submission = submission.drop('posts', axis = 1)

In [ ]:
submission['posts'] = test 
submission

,type,posts
0,ENFP,get accept ya bite well stop important open lo...
1,ENTP,offer rebel something war people friend block ...
2,INTJ,soulmates contradiction easy basic recurrence ...
3,INTJ,run nature q test let sound sarcastically irri...
4,INTJ,hour fast fast suspend see strict wampum eight...
...,...,...
9332,ENTP,increase thing liquor analyze partner consider...
9333,ENTP,senior vote wide shame learn talk problem broo...
9334,INTP,want relate top understand think read generate...
9335,ENFP,alot set intjs soooo alone ease cherish anothe...


In [ ]:
submission.to_csv('svm_1.csv', index=False)

In [ ]:
submission

,type,posts
0,ENFP,get accept ya bite well stop important open lo...
1,ENTP,offer rebel something war people friend block ...
2,INTJ,soulmates contradiction easy basic recurrence ...
3,INTJ,run nature q test let sound sarcastically irri...
4,INTJ,hour fast fast suspend see strict wampum eight...
...,...,...
9332,ENTP,increase thing liquor analyze partner consider...
9333,ENTP,senior vote wide shame learn talk problem broo...
9334,INTP,want relate top understand think read generate...
9335,ENFP,alot set intjs soooo alone ease cherish anothe...


제출 결과 test dataset에서 accuracy 96%를 보였습니다.

이 결과로 본선진출에 성공할 수 있었습니다.

# End